In [ ]:
### TODO: train baseline models (KNN recommender and/or random recommender) and report result

In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/CS247/Models')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
!pip install surprise
import surprise
from surprise import *
from util import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

     |████████████████████████████████| 11.8MB 24.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617614 sha256=61daff9a22ebdf7129b422b792910ae50853651297aa991ff7bb8963110704eb
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
train_data = pd.read_pickle("/content/drive/MyDrive/CS247/Data/full_training_data.pkl")[['review_profilename','beer_beerid','review_overall']]
train_data = merge_user_id(train_data, on='review_profilename').rename(columns={'id':'user_id'})[['user_id','beer_beerid','review_overall']]
test_data = pd.read_pickle("/content/drive/MyDrive/CS247/Data/test_data.pkl")[['review_profilename','beer_beerid','review_overall']]
test_data = merge_user_id(test_data, on='review_profilename').rename(columns={'id':'user_id'})[['user_id','beer_beerid','review_overall']]

In [ ]:
train_data.head()

,user_id,beer_beerid,review_overall
0,0,28577,4.0
1,0,45205,3.5
2,0,3977,4.0
3,0,1733,3.5
4,0,48166,4.5


In [ ]:
## https://surprise.readthedocs.io/en/stable/FAQ.html
reader = Reader(rating_scale=(0,5))
training_dataset = Dataset.load_from_df(train_data, reader)
training_set = training_dataset.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': True}
model = KNNBaseline(sim_options=sim_options)
model.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
reader = Reader(rating_scale=(0,5))
test_dataset = Dataset.load_from_df(test_data, reader)
testset = test_dataset.build_full_trainset()
testset = testset.build_testset()
predictions = model.test(testset)

surprise.accuracy.rmse(predictions, verbose=True)

RMSE: 0.6019


0.601917327532018

In [ ]:
from collections import defaultdict
## https://surprise.readthedocs.io/en/stable/FAQ.html
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print("precision at 5: ", sum(prec for prec in precisions.values()) / len(precisions))
print("recall at 5:", sum(rec for rec in recalls.values()) / len(recalls))

precision at 5:  0.6384590281997705
recall at 5: 0.3804149857215078


### Random Recommender

In [ ]:
from surprise import AlgoBase
import random

class RandomRecommender(AlgoBase):
  def __init__(self):
    AlgoBase.__init__(self)

  def estimate(self, u, i):
    return random.uniform(0,5)

In [ ]:
random_recommender = RandomRecommender()
random_recommender.fit(training_set)
random_predictions = random_recommender.test(testset)
surprise.accuracy.rmse(random_predictions, verbose=True)

RMSE: 2.0769


2.076930998748725

In [ ]:
precisions, recalls = precision_recall_at_k(random_predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print("precision at 5: ", sum(prec for prec in precisions.values()) / len(precisions))
print("recall at 5:", sum(rec for rec in recalls.values()) / len(recalls))

precision at 5:  0.3843753516530482
recall at 5: 0.1551772745055088
